# Isolation Similarity


<div style="text-align: right">
<b>
"Two points in a sparse region are more similar than two points of equal inter-point distance in a dense region"
</b>
</div>

<div style="text-align: right">
[arXiv:1907.00378v1]
</div>

In this work I implemented Isolation Kernel (Similarity) using iForest and aNNE. Isolation Kernel is described in the article "Nearest-Neighbour-Induced Isolation Similarity and Its Impact on Density-Based Clustering" [arXiv:1907.00378v1](https://arxiv.org/pdf/1907.00378.pdf). I used datasets "jain" and "pathbased" from [cs.uef.fi](http://cs.uef.fi/sipu/datasets) and "Breast Cancer Wisconsin (Diagnostic)" from [archive.ics.uci.edu](https://archive.ics.uci.edu/ml/datasets).
Algorithm MBSCAN based on iForest- and aNNE-Dissimilarity are compared with DBSCAN, SpectralClustering, AgglomerativeClustering.



In [89]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [2]:
# download datasets

! wget http://cs.uef.fi/sipu/datasets/jain.txt

! wget http://cs.uef.fi/sipu/datasets/pathbased.txt

! wget -r -nH --cut-dirs=2 -np -R "index.html*" https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/

clear_output()

To begin with, let's load datasets and normalize them using the min-max normalisation so that each attribute is in [0, 1] before the experiments begin.

In [81]:
from sklearn.preprocessing import MinMaxScaler

def load_wdbc_dataset():
    dataset = pd.read_csv('breast-cancer-wisconsin/wdbc.data', header=None, index_col=0)

    data = dataset.drop(dataset.columns[0], axis=1)
    target = dataset[dataset.columns[0]].replace('B', 1).replace('M', 0)

    # min-max scaling
    data = pd.DataFrame(MinMaxScaler().fit_transform(data), data.index)
    
    return data, target


def load_jain_dataset():
    dataset = open('jain.txt').read().splitlines()
    dataset = pd.DataFrame(list(map(lambda x: list(map(float, x.split('\t'))), dataset)))

    data = dataset.drop(dataset.columns[-1], axis=1)
    target = dataset[dataset.columns[-1]].astype(int).replace(2, 0)

    # min-max scaling
    data = pd.DataFrame(MinMaxScaler().fit_transform(data), data.index)

    return data, target


def load_pathbased_dataset():
    dataset = open('pathbased.txt').read().splitlines()
    dataset = pd.DataFrame(list(map(lambda x: list(map(float, x.split('\t'))), dataset)))

    data = dataset.drop(dataset.columns[-1], axis=1)
    target = dataset[dataset.columns[-1]].astype(str).replace('1.0', 2).replace('2.0', 1).replace('3.0', 0)

    # min-max scaling
    data = pd.DataFrame(MinMaxScaler().fit_transform(data), data.index)

    return data, target

Dataset "Breast Cancer Wisconsin (Diagnostic)" has 30 features and 569 objects which are devided by 2 classes.

In [5]:
wdbc_dat, wdbc_tar = load_wdbc_dataset()
display(wdbc_dat.head())

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,,,,,,,,,,,,,,,,,,,,,
842302,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
842517,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
84300903,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
84348301,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
84358402,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


Dataset "jain" has 2 features and 373 objects devided by 2 classes.

In [7]:
jain_dat, jain_tar = load_jain_dataset()
display(jain_dat.head())

,0,1
0,0.002466,0.582329
1,0.000000,0.508032
2,0.062885,0.502008
3,0.110974,0.451807
4,0.102343,0.510040


Dataset "pathbased" has 2 features and 300 objects devided by 3 classes.

In [166]:
pathbased_dat, pathbased_tar = load_pathbased_dataset()
display(pathbased_dat.head())

,0,1
0,0.231041,0.049822
1,0.220459,0.037367
2,0.181658,0.076512
3,0.179894,0.074733
4,0.156966,0.112100


In order to build Isolation Kernel, first, we need to select subsample $D_i'$ of sample $D'$ randomly where $i = 1 \dots t$ and $|D_i'| = \psi$.
Secondly, we denote the set of all partitions $H_i$ that are admissible under $D$ where each isolating partition $\Theta \in H_i$ isolates one data point from the rest of the points in a random subset $D_i' \in D$.
Thirdly, for all pairs $x, y$ of points from the sample we define value
$$ K_{\psi}(x, y | D) = \frac{1}{t}\sum_{i=1}^t I(x, y \in \Theta | \Theta \in H_i) $$
as an element of Isolation Kernel matrix.

Following on from this, let's define Isolation Dissimilarity as $p(x, y) = 1 − K_{\psi}(x, y)$.

Now let's realize the function that builds iForest-Dissimilarity.
The autors of the article reveal two shortcomings in using isolation trees.
The first one is that each isolation tree employs axis-parallel splits.
The second one is an imbalanced tree.
Some partitions are always overextended for the first few splits close to the root of an imbalanced tree and these are manifested as elongated rectangles.

In [9]:
from sklearn.ensemble import IsolationForest


def get_iForestDissimilarity(dataset, psi, t):
    similarity = []

    for _ in range(t):
        # generate random subset
        subset = np.random.choice(range(dataset.shape[0]), psi)

        # fit iForest on the generated subset
        iForest = IsolationForest(n_estimators=1)
        iForest.fit(dataset.iloc[subset])

        # for each object get its leaf
        leaves = iForest.estimators_[0].apply(dataset)

        # produce isolation kernel
        K = np.array([int(leaves[i] == leaves[j]) for i in range(len(leaves)) for j in range(len(leaves))])
        
        similarity.append(K.reshape((len(leaves), len(leaves))))

    return 1 - np.mean(np.array(similarity), axis=0)

Then let's realize the function that builds aNNE-Dissimilarity.
The partitions here are cells in Voronoi diagram.

In [11]:
def get_aNNEDissimilarity(dataset, psi, distance_matrix, t):
    similarity = []

    for _ in range(t):
        # generate random subset
        subset = np.random.choice(range(dataset.shape[0]), psi)

        # calculate distance to each cell's center
        cells = np.argmin(distance_matrix[subset], axis=0)

        # produce isolation kernel
        K = np.array([int(cells[i] == cells[j]) for i in range(len(cells)) for j in range(len(cells))])
        
        similarity.append(K.reshape((len(cells), len(cells))))

    return 1 - np.mean(np.array(similarity), axis=0)

Here is function of computing distance matrix, we will use it later.

In [12]:
def get_distance_matrix(data):
    distance_matrix = np.zeros((data.shape[0], data.shape[0]))

    for i in range(data.shape[0]):
        for j in range(data.shape[0]):
            distance_matrix[i, j] = np.linalg.norm(data.values[i] - data.values[j])

    return distance_matrix


Now it's time to use computed Dissimilarities.
For this we apply algorithm MBSCAN, which is DBSCAN that we fit on Isolation Dissimilarity matrix with metric equal to "precomputed".

In [76]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import f1_score


def grid_search(data, target, t=200):
    """
    eps    in [0.01, ..., 0.99]
    minPts in [2, ..., 40]
    psi    in [10 values from 2 to len(data) // 2]
    """

    dbscan, mbscan_iForest, mbscan_aNNE = [], [], []

    # generate list of params
    eps_minPts_list = [(eps, minPts) for eps in np.linspace(0, 1, 101)[1:-1] for minPts in range(2, 41)]
    psi_list = np.linspace(2, data.shape[0] // 2, 10).astype(int)

    # calculate distance matrix
    distance_matrix = get_distance_matrix(data)

    # calculate dissimilarities for each psi from psi_list
    iForestDissimilarities, aNNEDissimilarities = [], []
    for psi in tqdm(psi_list, desc='Calculating of isolation dissimilarities'):
        iForestDissimilarities.append(get_iForestDissimilarity(data, psi=psi, t=t))
        aNNEDissimilarities.append(get_aNNEDissimilarity(data, psi=psi, distance_matrix=distance_matrix, t=t))

    # searching for clustering params
    for eps, minPts in tqdm(eps_minPts_list, desc='Searching for clustering params'):
        # clusterize dataset using DBSCAN
        preds = DBSCAN(eps=eps, min_samples=minPts).fit_predict(data)
        score = f1_score(target, preds, average='macro', labels=np.unique(target))
        dbscan.append([round(score, 3), eps, minPts, '-'])

        for psi, iForestDissimilarity, aNNEDissimilarity in zip(psi_list, iForestDissimilarities, aNNEDissimilarities):
                # clusterize dataset using MBSCAN-iForest
                preds = DBSCAN(eps=eps, min_samples=minPts, metric='precomputed').fit_predict(iForestDissimilarity)
                score = f1_score(target, preds, average='macro', labels=np.unique(target))
                mbscan_iForest.append([round(score, 3), eps, minPts, psi])

                # clusterize dataset using MBSCAN-aNNE
                preds = DBSCAN(eps=eps, min_samples=minPts, metric='precomputed').fit_predict(aNNEDissimilarity)
                score = f1_score(target, preds, average='macro', labels=np.unique(target))
                mbscan_aNNE.append([round(score, 3), eps, minPts, psi])

    best = [max(clr, key=lambda x: x[0]) for clr in [dbscan, mbscan_iForest, mbscan_aNNE]]
    result = pd.DataFrame(data=best,
                          columns=['F1', 'eps', 'min_samples', 'psi'],
                          index=['dbscan', 'mbscan_iForest', 'mbscan_aNNE'])
    return result

In [19]:
wdbc_res = grid_search(wdbc_dat, wdbc_tar, t=200)

Searching of clustering params: 100%|██████████| 3861/3861 [17:55<00:00,  3.59it/s]


In [21]:
jain_res = grid_search(jain_dat, jain_tar, t=200)

Searching of clustering params: 100%|██████████| 3861/3861 [12:03<00:00,  5.34it/s]


In [77]:
pathbased_res = grid_search(pathbased_dat, pathbased_tar, t=200)

Searching of clustering params: 100%|██████████| 3861/3861 [08:48<00:00,  7.30it/s]


This table shows the result of dataset "wdbc" clustering by algorithms dbscan, mbscan_iForest and mbscan_aNNE.
For quality assessments, the F1 metric with "macro" averange is used.

In [51]:
wdbc_res

,F1,eps,min_samples,psi
dbscan,0.577,0.40,13,-
mbscan_iForest,0.906,0.18,33,2
mbscan_aNNE,0.930,0.67,17,33


This table shows the result of dataset "jain" clustering.

In [22]:
jain_res

,F1,eps,min_samples,psi
dbscan,0.828,0.12,23,-
mbscan_iForest,1.000,0.31,2,22
mbscan_aNNE,1.000,0.40,2,22


This table shows the result of dataset "pathbased" clustering.

In [78]:
pathbased_res

,F1,eps,min_samples,psi
dbscan,0.699,0.07,9,-
mbscan_iForest,0.747,0.51,13,34
mbscan_aNNE,0.988,0.80,7,100


So, let's try to clusterize our datasets by other clustering algorithms, for example, SpectralClustering and AgglomerativeClustering.

In [276]:
from sklearn.cluster import SpectralClustering, AgglomerativeClustering


def clustering(data, target):
    result = []
    target = target.astype(str).replace('2', 1).replace('1', 2).astype(int) if len(np.unique(target)) == 3 else target
    
    # clusterize dataset using SpectralClustering
    preds = SpectralClustering(n_clusters=len(np.unique(target))).fit_predict(data)
    score = f1_score(target, preds, average='macro', labels=np.unique(target))
    result.append([round(score, 3)])

    # clusterize dataset using AgglomerativeClustering
    preds = AgglomerativeClustering(n_clusters=len(np.unique(target))).fit_predict(data)
    score = f1_score(target, preds, average='macro', labels=np.unique(target))
    result.append([round(score, 3)])

    return pd.DataFrame(data=result,
                        columns=['F1'],
                        index=['spectral_clustering', 'agglomerative_clustering'])

In [295]:
wdbc_add_res = clustering(wdbc_dat, wdbc_tar)

In [285]:
jain_add_res = clustering(jain_dat, jain_tar)

In [369]:
pathbased_add_res = clustering(pathbased_dat, pathbased_tar)

In [395]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]


wdbc = pd.concat([wdbc_res[['F1']], wdbc_add_res])
wdbc['dataset'] = 'wdbc'

jain = pd.concat([jain_res[['F1']], jain_add_res])
jain['dataset'] = 'jain'

pathbased = pd.concat([pathbased_res[['F1']], pathbased_add_res])
pathbased['dataset'] = 'pathbased'

result = pd.concat([wdbc, jain, pathbased])
result.index.name = 'algorithm'

pd.pivot_table(result.reset_index(), values='F1', index='dataset', columns='algorithm').style.apply(highlight_max,axis=1)

algorithm,agglomerative_clustering,dbscan,mbscan_aNNE,mbscan_iForest,spectral_clustering
dataset,,,,,
jain,0.769000,0.828000,1.000000,1.000000,0.870000
pathbased,0.732000,0.699000,0.988000,0.747000,0.732000
wdbc,0.856000,0.577000,0.930000,0.906000,0.830000


As we see, the best algorithm on all datasets is algorithm based on Isolation Kernel – MBSCAN-aNNE.
The second algorithm in terms of quality is another one based on Isolation Kernel – MBSCAN-iForest.

To summarize, Isolation Similarity is indeed a realy good approach of Density-Based Clustering.